In [278]:
import torch

import copy
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

from torch import nn, optim

import torch.nn.functional as F
from arff2pandas import a2p


%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [279]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [328]:
df = pd.read_csv("LSTM2_A2_50.csv")
df.shape

(900, 20)

In [329]:
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,"6:22:4.602,A","6:22:7.779,B","6:22:10.892,C","6:22:13.185,D","6:22:13.986,E","6:22:15.966,A","6:22:18.587,B","6:22:20.445,C","6:22:20.892,D","6:22:24.903,E","6:22:26.144,A","6:22:29.35,B","6:22:31.875,C","6:22:33.717,D","6:22:34.842,E","6:22:36.288,A","6:22:36.705,B","6:22:38.18,C","6:22:40.031,D","6:22:42.104,E"
1,"22:04:18.1,A","22:04:18.221,B","22:04:19.236,C","22:04:19.627,D","22:04:21.59,E","22:04:22.671,A","22:04:24.732,B","22:04:29.498,C","22:04:29.965,D","22:04:30.515,E","22:04:32.629,A","22:04:32.818,B","22:04:36.936,C","22:04:40.394,D","22:04:45.233,E","22:04:48.398,A","22:04:48.839,B","22:04:53.012,C","22:04:55.705,D","22:04:57.492,E"
2,"22:45:30.422,A","22:45:33.664,B","22:45:34.951,C","22:45:38.179,D","22:45:41.667,E","22:45:44.853,A","22:45:46.256,B","22:45:50.213,C","22:45:52.318,D","22:45:56.6,E","22:45:56.778,A","22:46:1.4780000000000015,B","22:46:2.158,C","22:46:6.535,D","22:46:10.776,E","22:46:11.317,A","22:46:14.926,B","22:46:15.605,C","22:46:18.443,D","22:46:19.695,E"
3,"11:40:3.661,A","11:40:4.887,B","11:40:8.787,C","11:40:12.908,D","11:40:17.357,E","11:40:18.268,A","11:40:23.08,B","11:40:23.35,C","11:40:25.992,D","11:40:30.932,E","11:40:34.813,A","11:40:38.352,B","11:40:42.154,C","11:40:45.683,D","11:40:48.545,E","11:40:49.716,A","11:40:51.245,B","11:40:51.926,C","11:40:56.179,D","11:40:57.747,E"
4,"12:56:14.29,A","12:56:15.845,B","12:56:19.124,C","12:56:20.729,D","12:56:23.946,E","12:56:28.288,A","12:56:31.393,B","12:56:32.211,C","12:56:32.789,D","12:56:36.886,E","12:56:40.325,A","12:56:40.592,B","12:56:43.386,C","12:56:48.385,D","12:56:52.001,E","12:56:54.902,A","12:56:56.969,B","12:57:0.713000000000001,C","12:57:2.47,D","12:57:6.884,E"


In [330]:
df.tail()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
895,"21:17:38.357,A","2:58:32.962999999999994,A","14:00:9.775000000000006,B","1:47:31.535,A","9:12:43.339,E","23:06:17.063999999999993,D","13:14:11.570999999999998,C","23:51:58.738,E","3:27:24.988,C","2:55:9.397000000000006,B","2:20:48.182,E","18:07:30.857,B","11:32:40.594,A","7:33:12.799000000000007,C","13:33:44.083,A","22:01:55.711,C","13:28:22.914,C","7:18:9.587000000000003,A","1:55:7.9539999999999935,C","16:07:31.55,C"
896,"18:51:33.409,A","0:44:59.714,B","6:43:41.544,C","5:22:4.349999999999994,C","4:07:39.795,D","22:25:24.444000000000003,C","12:20:43.348,C","4:59:10.906999999999996,B","9:00:27.036,E","22:54:15.698999999999998,D","5:58:55.487,D","17:44:25.650000000000006,B","14:40:50.048,E","12:45:1.5180000000000007,C","7:43:19.619,B","16:22:51.733,B","6:58:45.584999999999994,C","4:51:7.4210000000000065,E","8:22:40.444,D","22:49:22.638000000000005,E"
897,"8:59:58.293,A","8:29:50.57599999999999,C","4:30:1.8220000000000027,A","16:23:36.385,D","4:45:52.143,E","3:12:4.8430000000000035,A","13:22:58.498,C","11:01:7.355000000000004,C","20:07:4.709000000000003,D","5:54:58.615,B","0:15:10.405000000000001,B","8:01:16.734,C","19:13:11.450999999999993,B","11:19:55.198,E","1:09:21.412999999999997,E","9:29:54.548,B","2:09:40.68600000000001,B","21:24:57.949,A","20:53:56.084,D","7:30:6.911000000000001,A"
898,"10:07:53.772,A","2:20:4.150999999999996,A","1:05:10.393,D","19:15:25.965,D","17:41:46.261,C","20:16:37.42700000000001,E","14:07:1.3220000000000027,B","22:35:21.28,C","7:39:47.616,A","10:29:4.9680000000000035,D","2:20:56.358,D","5:33:45.821,E","7:01:7.183000000000007,A","16:10:55.412,C","18:42:13.269000000000005,A","0:47:10.069000000000003,D","14:47:58.419,B","8:03:45.679,E","21:10:20.034999999999997,C","10:24:55.272,D"
899,"7:12:19.13,A","3:41:16.271,C","0:47:48.434,E","6:07:25.647000000000006,E","9:12:48.713,E","17:09:31.072000000000003,A","15:33:26.840000000000003,B","4:01:36.639,C","5:14:54.897,E","6:44:6.081999999999994,A","15:40:4.108000000000004,C","13:15:23.063,D","1:39:56.66,B","17:05:37.137,B","12:07:48.517,D","8:43:46.42100000000001,E","16:37:25.692999999999998,A","5:29:10.641000000000005,E","7:54:49.88,C","4:15:1.7819999999999965,C"


In [331]:

dic = {'A':'1','B':'2','C':'3','D':'4','E':'5','F':'6','G':'7','H':'8','I':'9','J':'0'}
t1 = ()

for colunm in range(len(df)):
    former = df.iloc[colunm,0].split(',')
    formerTime = pd.Timestamp(former[0])
    formerAction = former[1]
    for row in range(20):
        latter = df.iloc[colunm,row].split(',')
        latterTime = pd.Timestamp(latter[0])
        latterAction = latter[1]
        for key in dic.keys():
            if latterAction == key:
                latterAction = int(dic.get(key))
        timeStamp = latterTime - formerTime
        timeStampToSec = timeStamp.total_seconds()
        t1 = timeStampToSec, latterAction
        df.iloc[colunm,row] = t1
        formerTime = latterTime

In [332]:
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,"(0.0, 1)","(3.177, 2)","(3.113, 3)","(2.293, 4)","(0.801, 5)","(1.98, 1)","(2.621, 2)","(1.858, 3)","(0.447, 4)","(4.011, 5)","(1.241, 1)","(3.206, 2)","(2.525, 3)","(1.842, 4)","(1.125, 5)","(1.446, 1)","(0.417, 2)","(1.475, 3)","(1.851, 4)","(2.073, 5)"
1,"(0.0, 1)","(0.121, 2)","(1.015, 3)","(0.391, 4)","(1.963, 5)","(1.081, 1)","(2.061, 2)","(4.766, 3)","(0.467, 4)","(0.55, 5)","(2.114, 1)","(0.189, 2)","(4.118, 3)","(3.458, 4)","(4.839, 5)","(3.165, 1)","(0.441, 2)","(4.173, 3)","(2.693, 4)","(1.787, 5)"
2,"(0.0, 1)","(3.242, 2)","(1.287, 3)","(3.228, 4)","(3.488, 5)","(3.186, 1)","(1.403, 2)","(3.957, 3)","(2.105, 4)","(4.282, 5)","(0.178, 1)","(4.7, 2)","(0.68, 3)","(4.377, 4)","(4.241, 5)","(0.541, 1)","(3.609, 2)","(0.679, 3)","(2.838, 4)","(1.252, 5)"
3,"(0.0, 1)","(1.226, 2)","(3.9, 3)","(4.121, 4)","(4.449, 5)","(0.911, 1)","(4.812, 2)","(0.27, 3)","(2.642, 4)","(4.94, 5)","(3.881, 1)","(3.539, 2)","(3.802, 3)","(3.529, 4)","(2.862, 5)","(1.171, 1)","(1.529, 2)","(0.681, 3)","(4.253, 4)","(1.568, 5)"
4,"(0.0, 1)","(1.555, 2)","(3.279, 3)","(1.605, 4)","(3.217, 5)","(4.342, 1)","(3.105, 2)","(0.818, 3)","(0.578, 4)","(4.097, 5)","(3.439, 1)","(0.267, 2)","(2.794, 3)","(4.999, 4)","(3.616, 5)","(2.901, 1)","(2.067, 2)","(3.744, 3)","(1.757, 4)","(4.414, 5)"


In [333]:
df.tail()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
895,"(0.0, 1)","(-65945.394001, 1)","(39696.812001, 2)","(-43958.24, 1)","(26711.804, 5)","(50013.724999, 4)","(-35525.493, 3)","(38267.167001, 5)","(-73473.75, 3)","(-1935.591, 2)","(-2061.215, 5)","(56802.675, 2)","(-23690.263, 1)","(-14367.795, 3)","(21631.284, 1)","(30491.628, 3)","(-30812.797, 3)","(-22213.327, 1)","(-19381.633001, 3)","(51143.596001, 3)"
896,"(0.0, 1)","(-65193.695, 2)","(21521.83, 3)","(-4897.194001, 3)","(-4464.554999, 4)","(65864.649, 3)","(-36281.096, 3)","(-26492.441001, 2)","(14476.129001, 5)","(50028.662999, 4)","(-60920.211999, 4)","(42330.163, 2)","(-11015.602, 5)","(-6948.53, 3)","(-18101.899, 2)","(31172.114, 2)","(-33846.148001, 3)","(-7658.163999, 5)","(12693.023, 4)","(52002.194, 5)"
897,"(0.0, 1)","(-1807.717001, 3)","(-14388.753999, 1)","(42814.563, 4)","(-41864.242, 5)","(-5627.3, 1)","(36653.655, 3)","(-8511.143, 3)","(32757.354, 4)","(-51126.094, 2)","(-20388.21, 2)","(27966.329, 3)","(40314.716999, 2)","(-28396.252999, 5)","(-36633.785001, 5)","(30033.135001, 2)","(-26413.862, 2)","(69317.263, 1)","(-1861.865, 4)","(-48229.173, 1)"
898,"(0.0, 1)","(-28069.621001, 1)","(-4493.757999, 4)","(65415.572, 4)","(-5619.704, 3)","(9291.166, 5)","(-22176.105, 2)","(30499.958, 3)","(-53733.664, 1)","(10157.352, 4)","(-29288.61, 4)","(11569.463, 5)","(5241.362, 1)","(32988.229, 3)","(9077.857, 1)","(-64503.2, 4)","(50448.35, 2)","(-24252.74, 5)","(47194.355999, 3)","(-38724.762999, 4)"
899,"(0.0, 1)","(-12662.859, 3)","(-10407.837, 5)","(19177.213, 5)","(11123.066, 5)","(28602.359, 1)","(-5764.232, 2)","(-41510.201, 3)","(4398.258, 5)","(5351.184999, 1)","(32158.026001, 3)","(-8681.045, 4)","(-41726.403, 2)","(55540.477, 2)","(-17868.62, 4)","(-12242.096, 5)","(28419.271999, 1)","(-40095.051999, 5)","(8739.239, 3)","(-13188.098001, 3)"


In [334]:
train_df, val_df = train_test_split(
  df,
  test_size=0.15,
  random_state=RANDOM_SEED
)

val_df, test_df = train_test_split(
  val_df,
  test_size=0.33, 
  random_state=RANDOM_SEED
)

In [335]:
def create_dataset(df):
    sequences = df.to_numpy().tolist()
    dataset = [torch.tensor(s).unsqueeze(1).float() for s in sequences]
    n_seq, seq_len, m, n_features = torch.stack(dataset).shape
    
    return dataset, seq_len, m, n_features

In [336]:
train_dataset, seq_len, m, n_features = create_dataset(train_df)
val_dataset, _, _, _ = create_dataset(val_df)

In [337]:
class Encoder(nn.Module):

  def __init__(self, seq_len, n_features, embedding_dim=64):
    super(Encoder, self).__init__()

    self.seq_len, self.n_features = seq_len, n_features
    self.embedding_dim, self.hidden_dim = embedding_dim, 2 * embedding_dim

    self.rnn1 = nn.LSTM(
      input_size=2,
      hidden_size=self.hidden_dim,
      num_layers=1,
      batch_first=True
    )
    
    self.rnn2 = nn.LSTM(
      input_size=self.hidden_dim,
      hidden_size=embedding_dim,
      num_layers=1,
      batch_first=True
    )

  def forward(self, x):
    x = x.reshape((1, self.seq_len, -1))
    x, (_, _) = self.rnn1(x)
    x, (hidden_n, _) = self.rnn2(x)

    return hidden_n.reshape((1, self.embedding_dim))

In [338]:
class Decoder(nn.Module):

  def __init__(self, seq_len, input_dim=64, n_features=2):
    super(Decoder, self).__init__()

    self.seq_len, self.input_dim = seq_len, input_dim
    self.hidden_dim, self.n_features = 1 * input_dim, n_features

    self.rnn1 = nn.LSTM(
      input_size=input_dim,
      hidden_size=input_dim,
      num_layers=1,
      batch_first=True
    )

    self.rnn2 = nn.LSTM(
      input_size=input_dim,
      hidden_size=self.hidden_dim,
      num_layers=1,
      batch_first=True
    )

    self.output_layer = nn.Linear(self.hidden_dim, n_features)

  def forward(self, x):
    x = x.repeat(self.seq_len, self.n_features)
    x = x.reshape((self.n_features, self.seq_len, self.input_dim))

    x, (hidden_n, cell_n) = self.rnn1(x)
    x, (hidden_n, cell_n) = self.rnn2(x)
    x = x.reshape((-1, self.seq_len, self.hidden_dim))
    return self.output_layer(x)

In [339]:
class RecurrentAutoencoder(nn.Module):

  def __init__(self, seq_len, n_features, embedding_dim=64):
    super(RecurrentAutoencoder, self).__init__()

    self.encoder = Encoder(seq_len, n_features, embedding_dim).to(device)
    self.decoder = Decoder(seq_len, embedding_dim, n_features).to(device)

  def forward(self, x):
    x = self.encoder(x)
    x = self.decoder(x)

    return x

In [340]:
model = RecurrentAutoencoder(seq_len, n_features, 128)
model = model.to(device)

In [341]:
class F1_Loss(nn.Module):

    def __init__(self, epsilon=1e-1):
        super().__init__()
        self.epsilon = epsilon
        
    def forward(self, y_pred, y_true):
        y_true = y_true.flatten().to(torch.long)
        y_pred = y_pred[0].view(-1, 1)

        assert y_pred.ndim == 2
        assert y_true.ndim == 1
        y_true = F.one_hot(y_true, 10).to(torch.float64)
        y_pred = F.softmax(y_pred, dim=1)
        tp = (y_true * y_pred).sum(dim=0).to(torch.float64)
        tn = ((1 - y_true) * (1 - y_pred)).sum(dim=0).to(torch.float64)
        fp = ((1 - y_true) * y_pred).sum(dim=0).to(torch.float64)
        fn = (y_true * (1 - y_pred)).sum(dim=0).to(torch.float64)
        
        precision = tp / (tp + fp + self.epsilon)
        recall = tp / (tp + fn + self.epsilon)
        f1 = 2* (precision*recall) / (precision + recall + self.epsilon)
     
        '''
        f1 = f1.clamp(min=self.epsilon, max=1-self.epsilon)
        precision = precision.clamp(min=self.epsilon, max=1-self.epsilon)
        recall = recall.clamp(min=self.epsilon, max=1-self.epsilon)
        
        for i in range(len(y_pred_pc)):
            y_pred_pc[i] = round(y_pred_pc[i])
            
        precision_pc = precision_score(y_true_pc, y_pred_pc, average='macro')
        recall_pc = recall_score(y_true_pc, y_pred_pc, average='macro')
        '''
        
        return 1 - f1.mean(), 1 - precision.mean(), 1 - recall.mean()
f1_loss = F1_Loss().cuda()

In [342]:
def train_model(model, train_dataset, val_dataset, n_epochs):
  optimizer = torch.optim.Adam(model.parameters(), lr=1e-1)
  criterion = nn.MSELoss(reduction='mean').to(device)
  history = dict(train=[], val=[])

  best_model_wts = copy.deepcopy(model.state_dict())
  best_loss = 10000.0
  
  for epoch in range(1, n_epochs + 1):
    model = model.train()

    train_losses = []
    for seq_true in train_dataset:
      optimizer.zero_grad()
      seq_true = seq_true.view(-1,2)
      seq_true = seq_true.to(device)
      seq_pred = model(seq_true)
      loss = criterion(seq_pred, seq_true)

      loss.backward()
      optimizer.step()

      train_losses.append(loss.item())

    val_losses = []
    model = model.eval()
    with torch.no_grad():
      for seq_true in val_dataset:
        seq_true = seq_true.view(-1,2)
        seq_true = seq_true.to(device)
        seq_pred = model(seq_true)
        loss = criterion(seq_pred, seq_true)
        val_losses.append(loss.item())
        print(f1_loss(seq_pred, seq_true))

    train_loss = np.mean(train_losses)
    val_loss = np.mean(val_losses)

    history['train'].append(train_loss)
    history['val'].append(val_loss)

    if val_loss < best_loss:
      best_loss = val_loss
      best_model_wts = copy.deepcopy(model.state_dict())

    print(f'Epoch {epoch}: trainLoss {train_loss} / valLoss {val_loss}')

  model.load_state_dict(best_model_wts)
  return model.eval(), history

In [343]:
model, history = train_model(
  model, 
  train_dataset, 
  val_dataset, 
  n_epochs=1
)

C:\Anaconda3\envs\sequenceAnomaly\lib\site-packages\torch\nn\modules\loss.py:431: UserWarning: Using a target size (torch.Size([20, 2])) that is different to the input size (torch.Size([2, 20, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


(tensor(0.8446, dtype=torch.float64), tensor(0.9002, dtype=torch.float64), tensor(0.4098, dtype=torch.float64))
(tensor(0.8446, dtype=torch.float64), tensor(0.9002, dtype=torch.float64), tensor(0.4098, dtype=torch.float64))
(tensor(0.8444, dtype=torch.float64), tensor(0.9002, dtype=torch.float64), tensor(0.4096, dtype=torch.float64))
(tensor(0.8452, dtype=torch.float64), tensor(0.9002, dtype=torch.float64), tensor(0.4105, dtype=torch.float64))
(tensor(0.8461, dtype=torch.float64), tensor(0.9002, dtype=torch.float64), tensor(0.4119, dtype=torch.float64))
(tensor(0.8451, dtype=torch.float64), tensor(0.9002, dtype=torch.float64), tensor(0.4100, dtype=torch.float64))
(tensor(0.8445, dtype=torch.float64), tensor(0.9002, dtype=torch.float64), tensor(0.4097, dtype=torch.float64))
(tensor(0.8443, dtype=torch.float64), tensor(0.9002, dtype=torch.float64), tensor(0.4096, dtype=torch.float64))
(tensor(0.8440, dtype=torch.float64), tensor(0.9002, dtype=torch.float64), tensor(0.4095, dtype=torch.fl

RuntimeError: Class values must be non-negative.

In [ ]:
ax = plt.figure().gca()

ax.plot(history['train'])
ax.plot(history['val'])
plt.ylabel('Loss (Reconstruction Error)')
plt.xlabel('Epoch')
plt.legend(['train', 'test'])
plt.title('Loss over training epochs')
plt.show()

In [ ]:
for seq_true in val_dataset:
      seq_true = seq_true.view(-1,2)
      seq_true = seq_true.to(device)
      seq_pred = model(seq_true)

In [ ]:
class F1_Loss(nn.Module):

    def __init__(self, epsilon=1e-5):
        super().__init__()
        self.epsilon = epsilon
        
    def forward(self, y_pred, y_true):
        y_true = y_true.flatten().to(torch.long)
        y_pred = y_pred[0].view(-1, 1)
        y_true_pc = y_true
        y_pred_pc = y_pred.view(-1)
        assert y_pred.ndim == 2
        assert y_true.ndim == 1
        y_true = F.one_hot(y_true, 10).to(torch.float32)
        y_pred = F.softmax(y_pred, dim=1)
        tp = (y_true * y_pred).sum(dim=0).to(torch.float32)
        tn = ((1 - y_true) * (1 - y_pred)).sum(dim=0).to(torch.float32)
        fp = ((1 - y_true) * y_pred).sum(dim=0).to(torch.float32)
        fn = (y_true * (1 - y_pred)).sum(dim=0).to(torch.float32)
        
        precision = tp / (tp + fp + self.epsilon)
        recall = tp / (tp + fn + self.epsilon)
        f1 = 2* (precision*recall) / (precision + recall + self.epsilon)
        
        f1 = f1.clamp(min=self.epsilon, max=1-self.epsilon)
        precision = precision.clamp(min=self.epsilon, max=1-self.epsilon)
        recall = recall.clamp(min=self.epsilon, max=1-self.epsilon)

        y_true_pc = y_true_pc.detach().numpy()
        y_pred_pc = y_pred_pc.detach().numpy()

        for i in range(len(y_pred_pc)):
            y_pred_pc[i] = round(y_pred_pc[i])

        precision_pc = precision_score(y_true_pc, y_pred_pc, average='macro')
        recall_pc = recall_score(y_true_pc, y_pred_pc, average='micro')
        
        return 1 - f1.mean(), 1 - precision.mean(), 1 - recall.mean(), precision_pc, recall_pc
f1_loss = F1_Loss().cuda()